## Category-Level Funnel Segmentation

###### Goal
Understand which product categories lose profitability when high discounts are applied.

In [16]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import dataset

df = pd.read_csv("SampleSuperstore.csv")

In [3]:
# view head

df.head()

,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,22.3680,2,0.20,2.5164


In [4]:
# view dataset info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ship Mode     9994 non-null   object 
 1   Segment       9994 non-null   object 
 2   Country       9994 non-null   object 
 3   City          9994 non-null   object 
 4   State         9994 non-null   object 
 5   Postal Code   9994 non-null   int64  
 6   Region        9994 non-null   object 
 7   Category      9994 non-null   object 
 8   Sub-Category  9994 non-null   object 
 9   Sales         9994 non-null   float64
 10  Quantity      9994 non-null   int64  
 11  Discount      9994 non-null   float64
 12  Profit        9994 non-null   float64
dtypes: float64(3), int64(2), object(8)
memory usage: 1015.1+ KB


###### Data preparation

The data is already clean

###### Defining Funnel and Stages

* Total Orders
 * Discounted Orders
 * High Discounted Orders
 * Profitable High Discount Orders

In [17]:
category_funnel = (
    df.groupby('Category')
    .apply(lambda x: pd.Series({
        'Total Orders': x.shape[0],
        'Discounted Orders': x[x['Discount'] > 0].shape[0],
        'High Discount Orders': x[x['Discount'] >= 0.2].shape[0],
        'Profitable High Discount Orders': x[
            (x['Discount'] >= 0.2) & (x['Profit'] > 0)
        ].shape[0]
    }))
    .reset_index()
)

category_funnel

,Category,Total Orders,Discounted Orders,High Discount Orders,Profitable High Discount Orders
0,Furniture,2121,1285,1157,431
1,Office Supplies,6026,2897,2881,1994
2,Technology,1847,1014,1012,740


In [7]:
#conversion rate
category_funnel['Discount Conversion (%)'] = (
    category_funnel['Discounted Orders'] / category_funnel['Total Orders'] * 100
)

category_funnel['High Discount Conversion (%)'] = (
    category_funnel['High Discount Orders'] / category_funnel['Discounted Orders'] * 100
)

category_funnel['Profit Conversion (%)'] = (
    category_funnel['Profitable High Discount Orders'] / category_funnel['High Discount Orders'] * 100
)

category_funnel

,Category,Total Orders,Discounted Orders,High Discount Orders,Profitable High Discount Orders,Discount Conversion (%),High Discount Conversion (%),Profit Conversion (%)
0,Furniture,2121,1285,1157,431,60.584630,90.038911,37.251513
1,Office Supplies,6026,2897,2881,1994,48.075008,99.447705,69.212079
2,Technology,1847,1014,1012,740,54.899838,99.802761,73.122530


In [10]:
category_funnel.head()

,Category,Total Orders,Discounted Orders,High Discount Orders,Profitable High Discount Orders,Discount Conversion (%),High Discount Conversion (%),Profit Conversion (%)
0,Furniture,2121,1285,1157,431,60.584630,90.038911,37.251513
1,Office Supplies,6026,2897,2881,1994,48.075008,99.447705,69.212079
2,Technology,1847,1014,1012,740,54.899838,99.802761,73.122530


In [13]:
# melt category funnel data
category_funnel_long = category_funnel.melt(
    id_vars='Category',
    value_vars=[
        'Total Orders',
        'Discounted Orders',
        'High Discount Orders',
        'Profitable High Discount Orders'
    ],
    var_name='Stage',
    value_name='Orders'
)

category_funnel_long.head()

,Category,Stage,Orders
0,Furniture,Total Orders,2121
1,Office Supplies,Total Orders,6026
2,Technology,Total Orders,1847
3,Furniture,Discounted Orders,1285
4,Office Supplies,Discounted Orders,2897


In [14]:
# staging orders

stage_order = [
    'Total Orders',
    'Discounted Orders',
    'High Discount Orders',
    'Profitable High Discount Orders'
]

category_funnel_long['Stage'] = pd.Categorical(
    category_funnel_long['Stage'],
    categories=stage_order,
    ordered=True
)

In [15]:
# Visualizing

fig = px.funnel(
    category_funnel_long,
    x='Orders',
    y='Stage',
    color='Category',
    title='Category-wise Sales Funnel'
)

fig.show()

#### Interpretation
###### Furniture (High Risk Category)
* Highest discount usage (60.6%)
* Lowest profit conversion (37.3%)
* Heavy discounting destroys margins
* Indicates pricing sensitivity or high cost structure
* Action: Limit deep discounts on Furniture

###### Office Supplies (Stable Performer)
* Moderate discount usage
* High profit conversion (69.2%)
* Discounts are well absorbed

*  Action: Safe category for promotional campaigns

###### Technology (Best Performer)
* High discount conversion
* Highest profit conversion (73.1%)
* Strong margin resilience
*  Action: Use targeted high-discount promotions

###### Executive Insight 

Discount effectiveness varies significantly by product category. While Technology and Office Supplies maintain profitability under high discount levels, Furniture experiences substantial margin erosion. A uniform discount strategy is therefore suboptimal and should be replaced with category-specific pricing policies.